# Train

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import torch
import apex
import os
import logging

from pytorch_transformers import BertTokenizer
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor

Let's first:

1. Assign our paths
2. Check gpu status
3. Import our labels
4. Create a logger object
5. Decide the metrics we want to report

In [2]:
BASE = Path('data/phenotype_classification/')

DATA_PATH = BASE/'transformer/original/' # change this as necessary
LABEL_PATH = BASE
OUTPUT_DIR = BASE/'transformer/original/output' # change this as necessary
OUTPUT_DIR.mkdir(exist_ok=True)

In [3]:
# check if (multiple) GPUs are available

multi_gpu=False

if torch.cuda.is_available():
    
    device_cuda = torch.device("cuda")
    
    if torch.cuda.device_count() > 1:
        multi_gpu = True
else:
    device_cuda = torch.device("cpu")
    
print (multi_gpu)

True


In [4]:
categories = pd.read_csv(LABEL_PATH/'labels.csv', sep=',',header=None,names=["name"])
labels = list(categories['name'])
labels

['Advanced.Cancer',
 'Advanced.Heart.Disease',
 'Advanced.Lung.Disease',
 'Alcohol.Abuse',
 'Chronic.Neurological.Dystrophies',
 'Chronic.Pain.Fibromyalgia',
 'Dementia',
 'Depression',
 'Developmental.Delay.Retardation',
 'Non.Adherence',
 'Obesity',
 'Other.Substance.Abuse',
 'Schizophrenia.and.other.Psychiatric.Disorders',
 'Unsure']

In [5]:
logging.basicConfig(level=logging.NOTSET)
logger = logging.getLogger()

In [6]:
metrics = []
metrics.append({'name': 'accuracy_multilabel', 'function': accuracy_multilabel})
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})

## Pipeline

#### Create a DataBunch object

In [9]:
databunch = BertDataBunch(DATA_PATH, 
                          LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='val.csv',
                          text_col='text',
                          label_file='labels.csv',
                          label_col=labels,
                          batch_size_per_gpu=8,
                          max_seq_length=512,
                          multi_gpu=multi_gpu,
                          multi_label=True,
                          model_type='bert',
                          clear_cache=True)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0
INFO:pytorch_transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/aa5118/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:root:Writing example 0 of 1501
INFO:root:Saving features into cached file data/phenotype_classification/transformer/original/cache/cached_train_multi_label_512
INFO:root:Writing example 0 of 188
INFO:root:Saving features into cached file data/phenotype_classification/transformer/original/cache/cached_dev_multi_label_512


#### Create a Learner object

In [9]:
learner = BertLearner.from_pretrained_model(databunch,
                                            pretrained_path='bert-base-uncased',
                                            metrics=metrics,
                                            device=device_cuda,
                                            logger=logger,
                                            output_dir=OUTPUT_DIR,
                                            finetuned_wgts_path=None,
                                            warmup_steps=500,
                                            multi_gpu=multi_gpu,
                                            is_fp16=True,
                                            multi_label=True,
                                            logging_steps=20)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-config.json HTTP/1.1" 200 0
INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/aa5118/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:pytorch_transformers.modeling_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 14,
  "output_attentions": false,
  "output_hid

#### Train the model

In [10]:
learner.fit(epochs=10,
            lr=6e-5,
            validate=True, 	# Evaluate the model after each epoch
            schedule_type="warmup_cosine")

INFO:root:***** Running training *****
INFO:root:  Num examples = 1501
INFO:root:  Num Epochs = 20
INFO:root:  Total train batch size (w. parallel, distributed & accumulation) = 32
INFO:root:  Gradient Accumulation steps = 1
INFO:root:  Total optimization steps = 940


data/phenotype_classification/transformer/original/output/tensorboard
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/home/aa5118/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 20: 0.6848235229651133: 
INFO:root:eval_accuracy_multilabel after step 20: 0.026595744680851064: 
INFO:root:eval_accuracy_thresh after step 20: 0.5167173147201538: 
INFO:root:eval_roc_auc after step 20: 0.41002714468924506: 
INFO:root:eval_fbeta after step 20: 0.32542797923088074: 
INFO:root:lr after step 20: 2.4000000000000003e-06
INFO:root:train_loss after step 20: 0.715276351571083
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 40: 0.5737837851047516: 
INFO:root:eval_accuracy_multilabel after step 40: 0.0797872340425532: 
INFO:root:eval_accuracy_thresh after step 40: 0.822568416595459: 
INFO:root:eval_roc_auc after step 40: 0.5991413766728202: 
INFO:root:eval_fbeta after step 40: 0.3410872220993042: 
INFO:root:lr after step 40: 4.800000000000001e-06
INFO:root:train_loss after step 40: 0.6381511867046357
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


...

INFO:root:eval_loss after step 900: 0.3183351506789525: 
INFO:root:eval_accuracy_multilabel after step 900: 0.3191489361702128: 
INFO:root:eval_accuracy_thresh after step 900: 0.8920972943305969: 
INFO:root:eval_roc_auc after step 900: 0.791103788179408: 
INFO:root:eval_fbeta after step 900: 0.35263797640800476: 
INFO:root:lr after step 900: 1.2152107915650823e-06
INFO:root:train_loss after step 900: 0.08820373676717282
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 920: 0.3184267332156499: 
INFO:root:eval_accuracy_multilabel after step 920: 0.324468085106383: 
INFO:root:eval_accuracy_thresh after step 920: 0.8920972943305969: 
INFO:root:eval_roc_auc after step 920: 0.7912561895067373: 
INFO:root:eval_fbeta after step 920: 0.35287976264953613: 
INFO:root:lr after step 920: 3.053567435720195e-07
INFO:root:train_loss after step 920: 0.08838593736290931
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 940: 0.31818415721257526: 
INFO:root:eval_accuracy_multilabel after step 940: 0.324468085106383: 
INFO:root:eval_accuracy_thresh after step 940: 0.8928571343421936: 
INFO:root:eval_roc_auc after step 940: 0.7913374222897006: 
INFO:root:eval_fbeta after step 940: 0.35287976264953613: 
INFO:root:lr after step 940: 0.0
INFO:root:train_loss after step 940: 0.08576637730002404
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after epoch 20: 0.31818415721257526: 
INFO:root:eval_accuracy_multilabel after epoch 20: 0.324468085106383: 
INFO:root:eval_accuracy_thresh after epoch 20: 0.8928571343421936: 
INFO:root:eval_roc_auc after epoch 20: 0.7913374222897006: 
INFO:root:eval_fbeta after epoch 20: 0.35287976264953613: 


(940, 0.26950127923900774)

#### Save the model

In [11]:
learner.save_model()

## Wrap into one function

Let's make all the above into a function so we can call it multiple times easily and sequentially

In [7]:
BASE = Path('data/phenotype_classification/')
LABEL_PATH = BASE
BIOBERT_PATH = Path('biobert/')

def train(path_to_directory, model):
        
    DATA_PATH = BASE/path_to_directory
    OUTPUT_DIR = DATA_PATH/'output'/model 
    OUTPUT_DIR.mkdir(parents=True,exist_ok=True)
    
    if (model == "biobert"):
        tokenizer = BertTokenizer.from_pretrained(BIOBERT_PATH, 
                                                  do_lower_case=True)
        pretrained_path=BIOBERT_PATH
    elif (model == "bert"):
        tokenizer = "bert-base-uncased"
        pretrained_path="bert-base-uncased"
    else:
        print ("Model parameter must be either 'bert' or 'biobert'")
        return
    
    databunch = BertDataBunch(DATA_PATH, 
                              LABEL_PATH,
                              tokenizer=tokenizer,
                              train_file='train.csv',
                              val_file='val.csv',
                              text_col='text',
                              label_file='labels.csv',
                              label_col=labels,
                              batch_size_per_gpu=8,
                              max_seq_length=512,
                              multi_gpu=multi_gpu,
                              multi_label=True,
                              model_type='bert',
                              clear_cache=True)
    
    learner = BertLearner.from_pretrained_model(databunch,
                                                pretrained_path=pretrained_path,
                                                metrics=metrics,
                                                device=device_cuda,
                                                logger=logger,
                                                output_dir=OUTPUT_DIR,
                                                finetuned_wgts_path=None,
                                                warmup_steps=500,
                                                multi_gpu=multi_gpu,
                                                is_fp16=True,
                                                multi_label=True,
                                                logging_steps=20)
    
    learner.fit(epochs=10,
                lr=6e-5,
                validate=True, # Evaluate the model after each epoch
                schedule_type="warmup_cosine")
    
    learner.save_model()
    
    return

Let's get training!

In [8]:
for directory in ['original','original_2x','synthetic','combined']:
    for model in ['biobert','bert']:
        train('transformer/'+directory, model)

INFO:pytorch_transformers.tokenization_utils:Model name 'biobert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming 'biobert' is a path or url to a directory containing tokenizer files.
INFO:pytorch_transformers.tokenization_utils:Didn't find file biobert/added_tokens.json. We won't load it.
INFO:pytorch_transformers.tokenization_utils:Didn't find file biobert/special_tokens_map.json. We won't load it.
INFO:pytorch_transformers.tokenization_utils:loading file None
INFO:pytorch_transformers.tokenization_utils:loading file None
INFO:pytorch_transformers.tokenization_utils:loading file biobert/vocab.tx

INFO:root:eval_loss after step 20: 0.6245470146338145: 
INFO:root:eval_accuracy_multilabel after step 20: 0.09574468085106383: 
INFO:root:eval_accuracy_thresh after step 20: 0.7481003403663635: 
INFO:root:eval_roc_auc after step 20: 0.570418039716361: 
INFO:root:eval_fbeta after step 20: 0.34148284792900085: 
INFO:root:lr after step 20: 2.4000000000000003e-06
INFO:root:train_loss after step 20: 0.654225566983223
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 40: 0.5243857502937317: 
INFO:root:eval_accuracy_multilabel after step 40: 0.09574468085106383: 
INFO:root:eval_accuracy_thresh after step 40: 0.8461246490478516: 
INFO:root:eval_roc_auc after step 40: 0.5837567502286021: 
INFO:root:eval_fbeta after step 40: 0.315360426902771: 
INFO:root:lr after step 40: 4.800000000000001e-06
INFO:root:train_loss after step 40: 0.5891541808843612
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after epoch 1: 0.48875992000102997: 
INFO:root:eval_accuracy_multilabel after epoch 1: 0.07446808510638298: 
INFO:root:eval_accuracy_thresh after epoch 1: 0.8867781162261963: 
INFO:root:eval_roc_auc after epoch 1: 0.5992463323039054: 
INFO:root:eval_fbeta after epoch 1: 0.3081028163433075: 
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 60: 0.4463123132785161: 
INFO:root:eval_accuracy_multilabel after step 60: 0.047872340425531915: 
INFO:root:eval_accuracy_thresh after step 60: 0.8867781162261963: 
INFO:root:eval_roc_auc after step 60: 0.6106404881443269: 
INFO:root:eval_fbeta after step 60: 0.3013382852077484: 
INFO:root:lr after step 60: 7.2e-06
INFO:root:train_loss after step 60: 0.5029118552803993
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


...

INFO:root:eval_loss after step 180: 0.3399626662333806: 
INFO:root:eval_accuracy_multilabel after step 180: 0.047872340425531915: 
INFO:root:eval_accuracy_thresh after step 180: 0.8867781162261963: 
INFO:root:eval_roc_auc after step 180: 0.6610328496747813: 
INFO:root:eval_fbeta after step 180: 0.0: 
INFO:root:lr after step 180: 2.16e-05
INFO:root:train_loss after step 180: 0.37570609897375107
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after epoch 4: 0.33833127717177075: 
INFO:root:eval_accuracy_multilabel after epoch 4: 0.047872340425531915: 
INFO:root:eval_accuracy_thresh after epoch 4: 0.8867781162261963: 
INFO:root:eval_roc_auc after epoch 4: 0.6603463248276139: 
INFO:root:eval_fbeta after epoch 4: 0.0: 
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 200: 0.34463849663734436: 
INFO:root:eval_accuracy_multilabel after step 200: 0.047872340425531915: 
INFO:root:eval_accuracy_thresh after step 200: 0.8867781162261963: 
INFO:root:eval_roc_auc after step 200: 0.6527981458797008: 
INFO:root:eval_fbeta after step 200: 0.11696664988994598: 
INFO:root:lr after step 200: 2.4e-05
INFO:root:train_loss after step 200: 0.37602600604295733
INFO:root:Running evaluation
INFO:root:  Num examples = 188
INFO:root:  Batch size = 32


INFO:root:eval_loss after step 220: 0.3385680715243022: 
INFO:root:eval_accuracy_multilabel after step 220: 0.047872340425531915: 
INFO:root:eval_accuracy_thresh after step 220: 0.8867781162261963: 
INFO:root:eval_roc_auc after step 220: 0.6561955452804473: 
INFO:root:eval_fbeta after step 220: 0.0: 
INFO:root:lr after step 220: 2.64e-05
INFO:root:train_loss after step 220: 0.3774724841117859


KeyboardInterrupt: 